In [7]:
##Data Collection
import eikon as ek
import pandas as pd
import os

eikon_key = ''
ek.set_app_key(eikon_key)

target_securities = [".CSI300", ".HSI"]
start_date = "2019-01-01"
end_date = "2020-01-01"

#Possible values are tick, minute, hour, daily, weekly, monthly, quarterly, yearly.
frequency = 'daily'

#Create a folder for datasets
if not os.path.exists('Raw_Datasets'):
    os.makedirs('Raw_Datasets')

#Retrieve data for each security and create a folder for each security
for security in target_securities:
    if not os.path.exists('Raw_Datasets/' + security):
        os.makedirs('Datasets/' + security)
    
    data = ek.get_timeseries(security, start_date=start_date, end_date=end_date)

    file_name = security + start_date[:4] + '-' + end_date[:4] + ".csv"
    data.to_csv('Datasets/' + security + '/' + file_name)

c:\Users\yfxx_\.conda\envs\MachineLearning\lib\site-packages\pandas\core\dtypes\cast.py:1134: RuntimeWarning: invalid value encountered in cast
  if (arr.astype(int) == arr).all():
c:\Users\yfxx_\.conda\envs\MachineLearning\lib\site-packages\pandas\core\dtypes\cast.py:1153: RuntimeWarning: invalid value encountered in cast
  if (arr.astype(int) == arr).all():


In [10]:
#Technical Analysis
import talib as ta

#Create a folder for technical analysis
if not os.path.exists('Technical_Analysis'):
    os.makedirs('Technical_Analysis')

#Retrieve data for each security and create a folder for each security
for security in target_securities:
    if not os.path.exists('Technical_Analysis/' + security):
        os.makedirs('Technical_Analysis/' + security)
    
    data = pd.read_csv('Datasets/' + security + '/' + security + start_date[:4] + '-' + end_date[:4] + ".csv")
    
    data['SMA'] = ta.SMA(data['CLOSE'], timeperiod=14)
    data['RSI'] = ta.RSI(data['CLOSE'], timeperiod=14)
    data['MACD'], data['MACD_SIGNAL'], data['MACD_HIST'] = ta.MACD(data['CLOSE'], fastperiod=12, slowperiod=26, signalperiod=9)
    data['ADX'] = ta.ADX(data['HIGH'], data['LOW'], data['CLOSE'], timeperiod=14)
    data['CCI'] = ta.CCI(data['HIGH'], data['LOW'], data['CLOSE'], timeperiod=14)
    data['ADOSC'] = ta.ADOSC(data['HIGH'], data['LOW'], data['CLOSE'], data['VOLUME'], fastperiod=3, slowperiod=10)
    
    file_name = security + start_date[:4] + '-' + end_date[:4] + ".csv"
    data.to_csv('Technical_Analysis/' + security + '/' + file_name)